In [3]:
# 导入配置加载器
from config_loader import load_config

# ==================== 加载配置 ====================
print("=" * 60)
print("加载配置文件")
print("=" * 60)

try:
    config = load_config()
    zhipuai_config = config.get_model_config('zhipuai')
    print(f"\n✅ 配置加载成功！")
    print(f"  模型: {zhipuai_config['model']}")
    print(f"  温度: {zhipuai_config['temperature']}")
except Exception as e:
    print(f"\n❌ 配置加载失败: {e}")
    print("\n请按照以下步骤配置:")
    print("1. 复制 .env.example 为 .env")
    print("   命令: cp .env.example .env")
    print("2. 编辑 .env 文件，填写 ZHIPUAI_API_KEY")
    print("3. 保存后重新运行此单元格")
    raise


加载配置文件
✅ 已加载配置文件: /Users/yuyansong/AiProject/Langchain/langchain-example/langchain/.env

✅ 配置加载成功！
  模型: glm-4.5v
  温度: 0.7


## 基础语法

In [4]:
from typing import TypedDict, List
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    """Agent 状态定义

    这是 Agent 在执行过程中维护的所有数据
    """
    # 必需字段
    messages: List[BaseMessage]  # 对话历史

    # 可选字段（后面会讲）
    user_name: str
    session_id: str

## 消息类型（messages）

In [5]:
from langchain_core.messages import (
    HumanMessage,    # 用户消息
    AIMessage,       # AI 消息
    SystemMessage,   # 系统消息
    ToolMessage      # 工具返回消息
)

# 消息列表示例
messages = [
    SystemMessage(content="你是一个助手"),
    HumanMessage(content="你好"),
    AIMessage(content="你好！有什么可以帮你？"),
    HumanMessage(content="天气怎么样？"),
    AIMessage(content="让我查询一下..."),
    ToolMessage(content="北京：晴天", tool_call_id="call_123"),
    AIMessage(content="北京今天是晴天")
]

## 添加自定义字段

In [6]:
from typing import TypedDict, List, Optional
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
    """带自定义字段的状态"""
    messages: List[BaseMessage]

    # 用户信息
    user_id: Optional[str]       # 用户ID
    user_name: Optional[str]     # 用户名

    # 会话信息
    session_id: str              # 会话ID
    created_at: str              # 创建时间

    # 业务数据
    order_id: Optional[str]      # 订单ID（客服场景）
    product_id: Optional[str]    # 产品ID

## Annotated 注解（高级）

In [7]:
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage
from langgraph.graph import add_messages

class AgentState(TypedDict):
    # add_messages：自动合并消息列表
    messages: Annotated[List[BaseMessage], add_messages]

    # 普通字段：直接替换
    user_name: str


# **add_messages 的作用：**
# 假设当前状态
current_state = {
    "messages": [
        HumanMessage(content="你好"),
        AIMessage(content="你好！")
    ]
}

# 新消息
new_messages = [
    HumanMessage(content="天气怎么样？")
]

# 使用 add_messages 注解
# 结果：自动追加，不是替换
result = {
    "messages": [
        HumanMessage(content="你好"),
        AIMessage(content="你好！"),
        HumanMessage(content="天气怎么样？")  # 追加
    ]
}

## 简单示例：记住用户名

In [9]:
"""
LangChain 1.0 示例：记住用户信息
"""

from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain_community.chat_models import ChatZhipuAI
from typing import TypedDict, List
from langchain_core.messages import BaseMessage


# 定义工具
@tool
def save_user_name(name: str) -> str:
    """保存用户名

    Args:
        name: 用户的名字
    """
    return f"已记住你的名字：{name}"

@tool
def get_user_name() -> str:
    """获取用户名"""
    # 注意：这个简化版本无法真正记忆
    # 真正的记忆需要在状态中维护
    return "请先告诉我你的名字"

# 创建模型和 Agent
# 初始化模型
model = ChatZhipuAI(**zhipuai_config)

agent = create_agent(
    model=model,
    tools=[save_user_name, get_user_name],
    system_prompt="""你是一个记忆助手。

当用户告诉你名字时，使用 save_user_name 工具保存。
当用户询问名字时，使用 get_user_name 工具查询。

重要：你要记住对话历史中的信息，不要每次都重新询问。"""
)

# 测试
print("🤖 记忆助手测试\n")

messages = []

# 轮次1：告知名字
print("【轮次1】")
user_input = "我叫张三"
print(f"👤 用户: {user_input}")

messages.append({"role": "user", "content": user_input})
result = agent.invoke({"messages": messages})
messages = result["messages"]

ai_response = messages[-1].content
print(f"🤖 AI: {ai_response}\n")

# 轮次2：询问名字
print("【轮次2】")
user_input = "我叫什么名字？"
print(f"👤 用户: {user_input}")

messages.append({"role": "user", "content": user_input})
result = agent.invoke({"messages": messages})
messages = result["messages"]

ai_response = messages[-1].content
print(f"🤖 AI: {ai_response}\n")

🤖 记忆助手测试

【轮次1】
👤 用户: 我叫张三
🤖 AI: 你好呀，张三！很高兴认识你～有什么我可以帮你的吗？

【轮次2】
👤 用户: 我叫什么名字？
🤖 AI: 我之前已经记住了你的名字是张三哦～所以你是叫张三哒！



## 持久化状态：使用文件存储

In [ ]:
"""
LangChain 1.0 示例：持久化记忆
使用 JSON 文件存储对话历史
"""

import os
import json
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from datetime import datetime


class MemoryManager:
    """记忆管理器"""

    def __init__(self, storage_file: str = "conversation_memory.json"):
        self.storage_file = storage_file

    def save_conversation(self, session_id: str, messages: list):
        """保存对话到文件"""
        # 读取现有数据
        data = self._load_data()

        # 转换消息为可序列化格式
        serialized_messages = []
        for msg in messages:
            serialized_messages.append({
                "type": msg.type,
                "content": msg.content,
                "timestamp": datetime.now().isoformat()
            })

        # 保存
        data[session_id] = {
            "messages": serialized_messages,
            "updated_at": datetime.now().isoformat()
        }

        with open(self.storage_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

    def load_conversation(self, session_id: str) -> list:
        """从文件加载对话"""
        data = self._load_data()

        if session_id not in data:
            return []

        # 转换回消息对象
        messages = []
        for msg_data in data[session_id]["messages"]:
            if msg_data["type"] == "human":
                messages.append(HumanMessage(content=msg_data["content"]))
            elif msg_data["type"] == "ai":
                messages.append(AIMessage(content=msg_data["content"]))

        return messages

    def _load_data(self) -> dict:
        """加载数据文件"""
        if not os.path.exists(self.storage_file):
            return {}

        try:
            with open(self.storage_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        except:
            return {}

    def list_sessions(self) -> list:
        """列出所有会话"""
        data = self._load_data()
        return list(data.keys())

# 创建工具
@tool
def get_current_time() -> str:
    """获取当前时间"""
    return datetime.now().strftime("%Y年%m月%d日 %H:%M:%S")

# 创建 Agent
model = ChatZhipuAI(**zhipuai_config)

agent = create_agent(
    model=model,
    tools=[get_current_time],
    system_prompt="你是一个智能助手，能记住用户之前说过的话。"
)

def main():
    memory = MemoryManager()

    print("🤖 持久化记忆助手")
    print("=" * 60)

    # 选择或创建会话
    sessions = memory.list_sessions()
    if sessions:
        print(f"\n现有会话: {', '.join(sessions)}")
        session_id = input("输入会话ID（或输入新ID创建）: ").strip()
    else:
        session_id = input("输入新会话ID: ").strip()

    # 加载历史对话
    messages = memory.load_conversation(session_id)

    if messages:
        print(f"\n✅ 已加载 {len(messages)} 条历史消息")
        print("最近的对话:")
        for msg in messages[-4:]:  # 显示最后4条
            role = "👤" if msg.type == "human" else "🤖"
            print(f"  {role} {msg.content[:50]}...")
    else:
        print(f"\n✅ 创建新会话: {session_id}")

    print("\n输入 'quit' 退出\n")

    # 对话循环
    while True:
        user_input = input("👤 你: ")

        if user_input.lower() == 'quit':
            # 保存对话
            memory.save_conversation(session_id, messages)
            print(f"\n✅ 对话已保存到会话 {session_id}")
            break

        if not user_input.strip():
            continue

        # 添加用户消息
        messages.append(HumanMessage(content=user_input))

        # 调用 agent
        result = agent.invoke({"messages": messages})
        messages = result["messages"]

        # 显示回复
        ai_response = messages[-1].content
        print(f"🤖 AI: {ai_response}\n")

        # 自动保存
        memory.save_conversation(session_id, messages)

if __name__ == "__main__":
    main()

## 记忆窗口管理

In [ ]:
def trim_messages(messages: list, max_messages: int = 20) -> list:
    """保留最近的 N 条消息

    Args:
        messages: 完整消息列表
        max_messages: 保留的最大消息数
    """
    if len(messages) <= max_messages:
        return messages

    # 始终保留系统消息
    system_messages = [msg for msg in messages if msg.type == "system"]
    other_messages = [msg for msg in messages if msg.type != "system"]

    # 保留最近的消息
    recent_messages = other_messages[-max_messages:]

    return system_messages + recent_messages

In [10]:
"""
摘要记忆：自动总结旧对话，节省 token
"""

from typing import List, Optional
from langchain_core.messages import BaseMessage, SystemMessage
from langchain_core.language_models import BaseChatModel


class ConversationSummarizer:
    """对话摘要器"""
    
    def __init__(
        self,
        model: BaseChatModel,
        summary_threshold: int = 50,
        messages_to_summarize: int = 40,
        summary_template: Optional[str] = None
    ):
        """
        初始化摘要器
        
        Args:
            model: LLM 模型
            summary_threshold: 触发摘要的消息数阈值
            messages_to_summarize: 需要摘要的消息数量
            summary_template: 自定义摘要提示词模板
        """
        self.model = model
        self.summary_threshold = summary_threshold
        self.messages_to_summarize = messages_to_summarize
        self.summary_template = summary_template or self._default_template()
    
    @staticmethod
    def _default_template() -> str:
        """默认摘要提示词模板"""
        return """请总结以下对话的关键信息，包括：
1. 用户的基本信息和偏好
2. 讨论的主要话题
3. 重要的事实和决定
4. 待办事项或未完成的任务

对话内容：
{conversation}

请用简洁的语言总结（200字以内）："""
    
    def _format_conversation(self, messages: List[BaseMessage]) -> str:
        """格式化对话为文本"""
        conversation_lines = []
        
        for msg in messages:
            # 跳过系统消息和工具消息
            if msg.type in ["system", "tool"]:
                continue
            
            role = "用户" if msg.type == "human" else "AI"
            content = msg.content.strip()
            
            if content:  # 只添加非空内容
                conversation_lines.append(f"{role}: {content}")
        
        return "\n".join(conversation_lines)
    
    def summarize(self, messages: List[BaseMessage]) -> str:
        """
        生成对话摘要
        
        Args:
            messages: 需要摘要的消息列表
            
        Returns:
            摘要文本
        """
        if not messages:
            return "暂无对话内容"
        
        # 格式化对话
        conversation = self._format_conversation(messages)
        
        if not conversation:
            return "暂无有效对话内容"
        
        # 生成摘要
        prompt = self.summary_template.format(conversation=conversation)
        
        try:
            summary = self.model.invoke(prompt).content
            return summary.strip()
        except Exception as e:
            print(f"⚠️ 摘要生成失败: {e}")
            return "摘要生成失败"
    
    def apply_summary(self, messages: List[BaseMessage]) -> List[BaseMessage]:
        """
        应用摘要策略：如果消息过多，自动生成摘要并压缩历史
        
        Args:
            messages: 完整消息列表
            
        Returns:
            压缩后的消息列表
        """
        if len(messages) <= self.summary_threshold:
            return messages
        
        # 分离系统消息
        system_messages = [msg for msg in messages if msg.type == "system"]
        non_system_messages = [msg for msg in messages if msg.type != "system"]
        
        # 确定需要摘要的消息数量
        num_to_summarize = min(self.messages_to_summarize, len(non_system_messages) - 10)
        
        if num_to_summarize <= 0:
            return messages
        
        # 生成摘要
        messages_to_summarize = non_system_messages[:num_to_summarize]
        summary = self.summarize(messages_to_summarize)
        
        # 构建新的消息列表
        summary_message = SystemMessage(content=f"📝 之前的对话摘要：\n{summary}")
        recent_messages = non_system_messages[num_to_summarize:]
        
        return system_messages + [summary_message] + recent_messages


# ==================== 使用示例 ====================
print("=" * 60)
print("摘要记忆示例")
print("=" * 60)

# 创建摘要器
summarizer = ConversationSummarizer(
    model=model,
    summary_threshold=50,
    messages_to_summarize=40
)

# 模拟长对话
from langchain_core.messages import HumanMessage, AIMessage

messages = [
    SystemMessage(content="你是一个智能助手"),
    HumanMessage(content="我叫张三，是一名软件工程师"),
    AIMessage(content="你好张三！很高兴认识你。"),
    HumanMessage(content="我想学习 Python"),
    AIMessage(content="Python 是一门很好的语言，适合初学者。"),
]

# 添加更多消息以触发摘要（模拟）
for i in range(25):
    messages.append(HumanMessage(content=f"问题 {i+1}"))
    messages.append(AIMessage(content=f"回答 {i+1}"))

print(f"\n当前消息数: {len(messages)}")

# 应用摘要
compressed_messages = summarizer.apply_summary(messages)

print(f"压缩后消息数: {len(compressed_messages)}")
print(f"\n摘要消息:")
for msg in compressed_messages:
    if msg.type == "system" and "摘要" in msg.content:
        print(f"  {msg.content[:100]}...")
        break

print("\n" + "=" * 60)

摘要记忆示例

当前消息数: 55
压缩后消息数: 16

摘要消息:
  📝 之前的对话摘要：
1. 用户基本信息：张三，软件工程师；偏好：想学习 Python。  
2. 主要话题：围绕 Python 学习的各类问题（如基础语法、工具使用等）展开交流。  
3. 重要事实...



## 实战项目 - 智能客服机器人

In [11]:
"""
实战项目：智能客服机器人（LangChain 1.0）

功能：
- 用户信息管理
- 订单查询
- 问题记录
- 跨会话记忆
- 对话历史导出
"""

import os
import json
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from datetime import datetime
from typing import Optional

# ==================== 数据管理 ====================

class CustomerDatabase:
    """客户数据库（模拟）"""

    def __init__(self):
        self.customers = {
            "user001": {
                "name": "张三",
                "level": "VIP",
                "phone": "138****1234",
                "email": "zhangsan@example.com"
            },
            "user002": {
                "name": "李四",
                "level": "普通会员",
                "phone": "139****5678",
                "email": "lisi@example.com"
            }
        }

        self.orders = {
            "user001": [
                {"order_id": "ORD001", "product": "iPhone 15", "status": "已发货", "date": "2025-01-20"},
                {"order_id": "ORD002", "product": "AirPods Pro", "status": "已送达", "date": "2025-01-15"}
            ],
            "user002": [
                {"order_id": "ORD003", "product": "MacBook Pro", "status": "已发货", "date": "2025-01-22"}
            ]
        }

    def get_customer(self, user_id: str) -> Optional[dict]:
        """获取客户信息"""
        return self.customers.get(user_id)

    def get_orders(self, user_id: str) -> list:
        """获取客户订单"""
        return self.orders.get(user_id, [])

class ConversationStorage:
    """对话存储"""

    def __init__(self, storage_file: str = "customer_service_history.json"):
        self.storage_file = storage_file

    def save(self, user_id: str, session_id: str, messages: list, metadata: dict = None):
        """保存对话"""
        data = self._load()

        if user_id not in data:
            data[user_id] = {}

        # 序列化消息
        serialized = []
        for msg in messages:
            serialized.append({
                "type": msg.type,
                "content": msg.content,
                "timestamp": datetime.now().isoformat()
            })

        data[user_id][session_id] = {
            "messages": serialized,
            "metadata": metadata or {},
            "updated_at": datetime.now().isoformat()
        }

        with open(self.storage_file, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

    def load(self, user_id: str, session_id: str) -> list:
        """加载对话"""
        data = self._load()

        if user_id not in data or session_id not in data[user_id]:
            return []

        messages = []
        for msg_data in data[user_id][session_id]["messages"]:
            if msg_data["type"] == "human":
                messages.append(HumanMessage(content=msg_data["content"]))
            elif msg_data["type"] == "ai":
                messages.append(AIMessage(content=msg_data["content"]))
            elif msg_data["type"] == "system":
                messages.append(SystemMessage(content=msg_data["content"]))

        return messages

    def list_sessions(self, user_id: str) -> list:
        """列出用户的所有会话"""
        data = self._load()
        if user_id not in data:
            return []
        return list(data[user_id].keys())

    def _load(self) -> dict:
        """加载数据"""
        if not os.path.exists(self.storage_file):
            return {}
        try:
            with open(self.storage_file, 'r', encoding='utf-8') as f:
                return json.load(f)
        except:
            return {}

# ==================== 工具定义 ====================

# 全局数据库实例
db = CustomerDatabase()

@tool
def get_customer_info(user_id: str) -> str:
    """获取客户信息

    Args:
        user_id: 客户ID，如 user001, user002
    """
    customer = db.get_customer(user_id)
    if not customer:
        return f"未找到客户 {user_id} 的信息"

    return f"""客户信息：
        姓名：{customer['name']}
        会员等级：{customer['level']}
        电话：{customer['phone']}
        邮箱：{customer['email']}"""

@tool
def query_orders(user_id: str) -> str:
    """查询客户订单

    Args:
        user_id: 客户ID
    """
    orders = db.get_orders(user_id)
    if not orders:
        return f"客户 {user_id} 暂无订单记录"

    result = f"客户 {user_id} 的订单记录：\n\n"
    for order in orders:
        result += f"订单号：{order['order_id']}\n"
        result += f"商品：{order['product']}\n"
        result += f"状态：{order['status']}\n"
        result += f"下单时间：{order['date']}\n"
        result += "-" * 40 + "\n"

    return result

@tool
def get_order_status(order_id: str) -> str:
    """查询订单状态

    Args:
        order_id: 订单号，如 ORD001
    """
    # 在所有订单中查找
    for user_id, orders in db.orders.items():
        for order in orders:
            if order['order_id'] == order_id:
                return f"""订单 {order_id} 状态：
                  商品：{order['product']}
                  状态：{order['status']}
                  下单时间：{order['date']}

                  {'预计明天送达' if order['status'] == '已发货' else ''}"""

    return f"未找到订单 {order_id}"

@tool
def create_ticket(user_id: str, problem: str) -> str:
    """创建客服工单

    当客户问题无法立即解决时，创建工单记录

    Args:
        user_id: 客户ID
        problem: 问题描述
    """
    ticket_id = f"TICKET_{datetime.now().strftime('%Y%m%d%H%M%S')}"

    # 模拟创建工单
    return f"""已为您创建工单：
      工单号：{ticket_id}
      问题：{problem}
      状态：待处理

      我们的工作人员会在 24 小时内联系您。感谢您的耐心等待！"""

@tool
def search_knowledge_base(keyword: str) -> str:
    """搜索知识库

    Args:
        keyword: 搜索关键词
    """
    # 模拟知识库
    knowledge = {
        "退货": "退货政策：\n1. 7天无理由退货\n2. 商品需保持完好\n3. 提供订单号和退货原因\n4. 联系客服办理",
        "发票": "发票申请：\n1. 登录账户\n2. 进入订单详情\n3. 点击【申请发票】\n4. 填写发票信息",
        "配送": "配送说明：\n1. 全国包邮（偏远地区除外）\n2. 一般3-5个工作日送达\n3. 可选择上门自提",
        "支付": "支付方式：\n1. 微信支付\n2. 支付宝\n3. 银行卡\n4. 货到付款（部分地区）"
    }

    for key, value in knowledge.items():
        if keyword in key:
            return value

    return f"未找到关于'{keyword}'的相关信息，请联系人工客服"

# ==================== 客服机器人 ====================

class CustomerServiceBot:
    """智能客服机器人"""

    def __init__(self):
        # 初始化模型
        self.model = ChatZhipuAI(**zhipuai_config)

        # 工具列表
        self.tools = [
            get_customer_info,
            query_orders,
            get_order_status,
            create_ticket,
            search_knowledge_base
        ]

        # 系统提示词
        system_prompt = """你是一个专业的客服助手。

          你的职责：
          1. 礼貌、友好地回答客户问题
          2. 使用工具查询订单、客户信息
          3. 解答常见问题（退货、发票、配送等）
          4. 无法解决时创建工单

          工作流程：
          1. 首先确认客户身份（询问客户ID或订单号）
          2. 了解客户问题
          3. 使用合适的工具查询信息或解决问题
          4. 用通俗易懂的语言回复客户
          5. 如果无法解决，创建工单

          注意事项：
          - 始终保持专业和礼貌
          - 回答要准确，不要编造信息
          - 如果不确定，建议联系人工客服
          - 记住对话历史中的客户信息

          请始终使用中文。"""

        # 创建 agent
        self.agent = create_agent(
            model=self.model,
            tools=self.tools,
            system_prompt=system_prompt
        )

        # 存储
        self.storage = ConversationStorage()

        # 当前会话
        self.current_user = None
        self.current_session = None
        self.messages = []

    def start_session(self, user_id: str, session_id: str = None):
        """开始会话"""
        self.current_user = user_id
        self.current_session = session_id or f"session_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

        # 加载历史对话
        self.messages = self.storage.load(user_id, self.current_session)

        if not self.messages:
            # 新会话，添加欢迎消息
            welcome = f"您好！我是智能客服助手。请问有什么可以帮您？\n\n提示：告诉我您的客户ID（如 user001），我可以为您查询订单和个人信息。"
            self.messages.append(AIMessage(content=welcome))

    def chat(self, user_input: str) -> str:
        """对话"""
        # 添加用户消息
        self.messages.append(HumanMessage(content=user_input))

        # 调用 agent
        result = self.agent.invoke({"messages": self.messages})
        self.messages = result["messages"]

        # 自动保存
        self.storage.save(self.current_user, self.current_session, self.messages)

        # 返回 AI 回复
        for msg in reversed(self.messages):
            if msg.type == "ai" and msg.content:
                return msg.content

        return "抱歉，我暂时无法处理您的请求。"

    def end_session(self):
        """结束会话"""
        self.storage.save(
            self.current_user,
            self.current_session,
            self.messages,
            metadata={"ended_at": datetime.now().isoformat()}
        )

# ==================== 主程序 ====================

def main():
    bot = CustomerServiceBot()

    print("=" * 70)
    print("🤖 智能客服机器人（LangChain 1.0）")
    print("=" * 70)

    # 用户身份
    print("\n请输入您的客户ID（测试账号：user001 或 user002）")
    user_id = input("客户ID: ").strip() or "user001"

    # 检查历史会话
    sessions = bot.storage.list_sessions(user_id)
    if sessions:
        print(f"\n您有 {len(sessions)} 个历史会话")
        print("会话列表:", ", ".join(sessions[-3:]))  # 显示最近3个
        choice = input("\n继续上次对话？(y/n): ").strip().lower()
        if choice == 'y':
            session_id = sessions[-1]  # 最近的会话
            print(f"✅ 继续会话: {session_id}")
        else:
            session_id = None
            print("✅ 开始新会话")
    else:
        session_id = None
        print(f"\n✅ 欢迎新客户: {user_id}")

    # 开始会话
    bot.start_session(user_id, session_id)

    # 显示欢迎消息
    if bot.messages:
        print(f"\n🤖 {bot.messages[-1].content}\n")

    print("输入 'quit' 结束对话\n")

    # 对话循环
    while True:
        user_input = input("👤 您: ")

        if user_input.lower() == 'quit':
            bot.end_session()
            print(f"\n✅ 对话已保存")
            print(f"会话ID: {bot.current_session}")
            print("感谢使用，再见！👋")
            break

        if not user_input.strip():
            continue

        # 调用客服机器人
        response = bot.chat(user_input)
        print(f"\n🤖 客服: {response}\n")

if __name__ == "__main__":
    main()

🤖 智能客服机器人（LangChain 1.0）

请输入您的客户ID（测试账号：user001 或 user002）

✅ 欢迎新客户: 12

🤖 您好！我是智能客服助手。请问有什么可以帮您？

提示：告诉我您的客户ID（如 user001），我可以为您查询订单和个人信息。

输入 'quit' 结束对话


🤖 客服: 
您好！我看到您输入了"111"，请问这是您的客户ID吗？如果是的话，我可以帮您查询相关信息；如果不是，您可以告诉我具体的问题，我会尽力为您提供帮助。


✅ 对话已保存
会话ID: session_20251208_231546
感谢使用，再见！👋
